In [1]:
import sys
sys.path.append(r"C:\Users\WilcoSievers\OneDrive - MOYO Business Advisory\Documents\GitHub\python-util")

In [2]:
import exutil
import pandas
import openpyxl
import numpy

In [3]:
base_path = r"C:\Users\WilcoSievers\OneDrive - MOYO Business Advisory\Documents\Assigments\2025-07-04 - Mine data analysis\Data\Combine data\KPI FILE 19 Oct 24"
file_pattern = "*.xlsx"
file_paths = exutil.filter_files(base_path, file_pattern)

In [6]:
sheet_targets = ["densities & reagents", "densities and reagents"];
start_points = ["SP"];
end_points = ["AG HT %CaF2", "HT AVG"]; 

In [33]:
def search_for_columns(file_paths : list[str], target_sheets : list[str], start_points : list , end_points : list[str] = ['NaN'], **kwargs):
    """
    Takes in 
    
    *Args:
        
        file_paths = List of file paths that need to be searched
        
        target_sheets = Sheet name to target, if no hit is recieved for first val continue to next val and search unit hit.
        
        start_points = List[str] of multiple values to search. These points are used as an anchor points for when selecting values. If the first value in the iterable returns None, move onto the next value.
        
        end_points = List[str] of multiple values to search. These points are used as an anchor points for when selecting values. If the first value in the iterable returns None, move onto the next value.

    **kwargs:
        
        start_pivot = Tuple of (int, int) as (row, col) - used to pivot from start point 
        
        end_pivot = Tuple of (int, int) as (row, col) - used to pivot from end point 
        
        searched_columns = Known columns of previous search results  
        
        searched_files = list[str] - list of files that have been searched and values that have
        
        start_default = (row, col) - Start value to default to if no resutls are found. If  'start_default' is not given, and the start points give no result, an error will be raised.
        
        end_default = (row, col) - Start value to default to if no resutls are found. If  'start_default' is not given, and the start points give no result, an error will be raised
        
        columns_to_exclue = Columns to remove, as there are certain cell values that are just thrown around in the bloody sheet.
    """
    # import dependant functions
    from openpyxl import load_workbook
    from exutil import is_similar, search_excl_val, exc_coord_to_rc, get_excl_sheet_vals, filter_files
    from datetime import datetime

    
    # Prelude variables used for various downstream processes
    start_point, end_point = {'row' : 0, 'col' : 0}, {'row' : 0, 'col' : 0};
    start_point_pivot, end_point_pivot = {'row' : 0, 'col' : 0}, {'row' : 0, 'col' : 0};
    start_point_default, end_point_default = {'row' : 0, 'col' : 0}, {'row' : 0, 'col' : 0};
    start_default, end_default = {'row' : None, 'col' : None}, {'row' : None, 'col' : None};
    
    column_result = [];
    column_source = [];
    searched_files = [];
    columns_to_exclue = [];

    
    for key, val in kwargs.items():
        match key:
            case 'start_pivot':
                start_point_pivot['row'], start_point_pivot['col'] = val;
            case 'end_pivot':
                end_point_pivot['row'], end_point_pivot['col'] = val;
            case 'searched_files':
                searched_files = searched_files + val; # append the files
            case 'start_default':
                start_point_default['row'], start_point_default['col'] = val;
            case 'end_default':
                end_point_default['row'], end_point_default['col'] = val;
            case 'searced_columns':
                column_result = searched_columns;
            case 'columns_to_exclue':
                columns_to_exclue = val;
            case _:
                raise NameError(f"Unkown values '{key}' - '{val}' fed. Expected **kwargs = [start_pivot, end_pivot, searched_files, start_default, end_default]")
    
    # remove file paths already searched
    files_paths_to_search = file_paths;


    # this is inneficient method. Search throygh file paths list and filter out values that have been searched
    for file_i in searched_files:
        file_slice_idx = file_i.rfind('\\') if file_i.rfind('\\')  != -1 else 0;
        files_paths_to_search = filter_files(files_paths_to_search,                    # the path list to iterate and make shrink
                                             '*' + file_i[file_slice_idx::] + '*',     # the pattern to search for, just look at final file
                                             matchPattern = False)
        
    for file_i in files_paths_to_search:
        try: 
            active_workbook = load_workbook(file_i);
        except PermissionError as pe:
            print(pe); # a file is open in another app, thus permission is denied
            continue;

        # Search for most relevenat sheet
        for sheet_ii in target_sheets:
            similar_sheets, simalr_sheet_scores = is_similar(sheet_ii, active_workbook.sheetnames, return_similarrity_score=True);
            if similar_sheets is not None:
                active_sheet_name = similar_sheets[simalr_sheet_scores.argmax()];
                break;

        # set active sheet
        active_sheet = active_workbook[active_sheet_name];

        
        # search for start anchor pivot point
        for start_pii in start_points:
            excl_coord, _ = search_excl_val(active_sheet, [start_pii], return_first_hit = True);
            if excl_coord is not None:
                start_point['row'], start_point['col'] =  exc_coord_to_rc(excl_coord);
                start_point['row'] = start_point['row'] + start_point_pivot['row']; 
                start_point['col'] = start_point['col'] + start_point_pivot['col']; 
                break;
        # Ensure start point does not have any errors
        if (start_point['row'] == 0) and (start_point['col'] == 0):
            if (start_point_default['row'] is None) and (start_point_default['col'] is None):
                raise UserWarning(f"Could not find start values '{start_points}' in '{active_sheet_name}' in '{file_i}'")
            else:
                start_point = start_point_default;

                
        # search for end anchor pivot point
        for end_pii in end_points:
            excl_coord, _ = search_excl_val(active_sheet, [end_pii], return_first_hit = True);

            if excl_coord is not None:
                end_point['row'], end_point['col'] =  exc_coord_to_rc(excl_coord);
                end_point['row'] += end_point_pivot['row']; 
                end_point['col'] += end_point_pivot['col']; 
                break;
        # Ensure end point does not have any errors
        if (end_point['row'] == 0) and (end_point['col'] == 0):
            if (end_point_default['row'] is None) and (end_point_default['col'] is None):
                raise UserWarning(f"Could not find end values '{end_points}' in '{active_sheet_name}' in '{file_i}'")
            else:
                end_point =  end_point_default;

        # extract the values, columns are in signle line, thus use start col as singular column
        extracted_columns = get_excl_sheet_vals(active_sheet,   # columns are in singular column
                                               (start_point['row'], start_point['col']),
                                               (end_point['row'], start_point['col']));

        # remove any None/ empty values
        extracted_columns = [colVal for colVal in extracted_columns if colVal]; 
        # rek0ve date values
        extracted_columns = [colVal for colVal in extracted_columns if not isinstance(colVal, datetime)]; 
        # remove any weird values
        extracted_columns = [colVal for colVal in extracted_columns if (str(colVal).find('=') == -1) and str(colVal).find(',') == -1];

        
        for new_col_ii in extracted_columns:
            if new_col_ii not in column_result:
                column_result.append(new_col_ii);
                column_source.append(file_i);

    # final cleanup for values
    col_source_list = [];
    col_list = [];
    for col_val_ii, path_source_ii in zip(column_result, column_source):
        if col_val_ii not in columns_to_exclue:
            col_source_list.append(path_source_ii[path_source_ii.rfind('\\')::]);   # just take the final column/ slice
            col_list.append(col_val_ii);
            
    return col_list, col_source_list;

In [ ]:
A,b = search_for_columns(  file_paths, sheet_targets, 
                           start_points, end_points,
                           start_pivot=(0,-3), end_pivot=(-1,0), 
                           searched_files =  previously_searched_files)

In [32]:
a =  'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2023\\Copy of Copy of Densities Reagent 27 Oct 2023.xlsx'

b = a.rfind("\\") if a.rfind("\\") != -1 else 0;
b

0

In [9]:
column_searched_files = ['C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2023\\Copy of Copy of Densities Reagent 27 Oct 2023.xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2023\\Copy of Copy of Densities Reagent 27 Oct 2023.xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2023\\Copy of Copy of Densities Reagent 27 Oct 2023.xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2023\\Copy of Copy of Densities Reagent 27 Oct 2023.xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2023\\Copy of Copy of Densities Reagent 27 Oct 2023.xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2023\\Copy of Copy of Densities Reagent 27 Oct 2023.xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2023\\Copy of Copy of Densities Reagent 27 Oct 2023.xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2023\\Copy of Copy of Densities Reagent 27 Oct 2023.xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2023\\Copy of Copy of Densities Reagent 27 Oct 2023.xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2023\\Copy of Copy of Densities Reagent 27 Oct 2023.xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2023\\Copy of Copy of Densities Reagent 27 Oct 2023.xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2023\\Copy of Copy of Densities Reagent 27 Oct 2023.xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2023\\Copy of Copy of Densities Reagent 27 Oct 2023.xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2023\\Copy of Copy of Densities Reagent 27 Oct 2023.xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2023\\Copy of Copy of Densities Reagent 27 Oct 2023.xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2023\\Copy of Copy of Densities Reagent 27 Oct 2023.xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2023\\Copy of Copy of Densities Reagent 27 Oct 2023.xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2023\\Copy of Copy of Densities Reagent 27 Oct 2023.xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2023\\Copy of Copy of Densities Reagent 27 Oct 2023.xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2023\\Copy of Copy of Densities Reagent 27 Oct 2023.xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2023\\Copy of Copy of Densities Reagent 27 Oct 2023.xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2023\\Copy of Copy of Densities Reagent 27 Oct 2023.xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2023\\Copy of Copy of Densities Reagent 27 Oct 2023.xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2023\\Copy of Copy of Densities Reagent 27 Oct 2023.xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2023\\Copy of Densities Reagent 06 july 2023.xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2023\\Copy of Densities Reagent 06 july 2023.xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2024\\01 KPI FILE-August 24.xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2024\\01 KPI FILE-August 24.xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2024\\01 KPI FILE-August 24.xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2024\\01 KPI FILE-August 24.xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2024\\01 KPI FILE-August 24.xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2024\\Copy of Copy of Copy of KPI FILE 07 July 2024.xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2024\\Copy of Copy of KPI FILE 19 Oct 24.xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2024\\Copy of Copy of KPI FILE 19 Oct 24.xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2024\\Copy of Copy of KPI FILE- 22 Sep 24.xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2024\\Copy of KPI FILE  13 NOV 2024.xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2024\\Copy of KPI FILE  13 NOV 2024.xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2024\\Copy of KPI FILE  13 NOV 2024.xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2024\\Copy of KPI FILE  13 NOV 2024.xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2024\\Densities Reagent  15 April 2024.xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2024\\Densities Reagent 02 May 2024.xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2025\\Copy of Copy of KPI FILE(09.06.2025) (1).xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2025\\Copy of Copy of KPI FILE(09.06.2025) (1).xlsx',
 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2025\\Copy of Copy of KPI FILE(09.06.2025) (1).xlsx']

In [11]:
know_column_list = [' 35% to 40% CaF2', 1163, 1164, 'CRT', 'ARF', 'CL1 Feed', 'CL2 Feed', 'CL2 concentrate', 'CL3 concentrate', 'CL4 concentrate', 'CL5 concentrate', 'Refloat feed', 'Refloat 1 concentrate', 'Refloat 2 concentrate', 'Refloat 3 concentrate', '(ml/min)', 'AR Collector ', 'CL1 Collector ', 'Collector (CCT) ', 'CL1 Dextrin / CL2', 'CL2 Dextrin ', 'CL3 Dextrin ', 'CL4 Dextrin ', 'Coagulant ', 'RF2 Dextrin', 'CL5 Dispersant', 'Surge Tank Density', 'WHIMS Feed Density', 'Tails Flocculant Flow(L/min)', 'Flocculant Strength', 'Soda ash flow', 'Metspar concentrate', 'Surge Tank Density(SCADA)', 'Conditioner 1(Manual)', 'Conditioner 1', 'FLOC WTP1', 'FLOC WTP2', 'Coag WTP1', 'Coag WTP2', 'Flocculant', 'Flocculant Dosing Strength', ' ', 'CL6 concentrate(SCADA)', 'AR Dextrin']